In [1]:
from keras.layers import Bidirectional
from keras.utils.np_utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
from nltk import WordNetLemmatizer, PorterStemmer, RegexpTokenizer
from nltk.corpus import stopwords
!pip install pydotplus

import tensorflow
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import seaborn as sns
from gensim.models import KeyedVectors
from keras import Input, Model, Sequential
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras_preprocessing.text import Tokenizer
#from tensorflow.python.keras.utils import to_categorical
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Embedding
#from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,GRU, Input
from keras.layers import Dense, Dropout
from keras.layers import Flatten, Attention, concatenate,Input
from sklearn.metrics import  confusion_matrix,f1_score,accuracy_score
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
warnings.filterwarnings('ignore')

path = '../data/'
train = pd.read_csv(path + 'dreaddit-train-processed.csv', encoding = "ISO-8859-1")
test = pd.read_csv(path + 'dreaddit-test-processed.csv', encoding = "ISO-8859-1")


You should consider upgrading via the 'c:\users\sichi\pycharmprojects\nlpproiect\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
features_train = train.drop(['subreddit','post_id','sentence_range','text','id','label','confidence','social_timestamp','social_karma','syntax_ari','processed_text'], 1)
features_test = test.drop(['subreddit','post_id','sentence_range','text','id','label','confidence','social_timestamp','social_karma','syntax_ari','processed_text'], 1)

print(features_train.columns)
features_len = len(features_train.columns)

Index(['Unnamed: 0', 'lex_liwc_WC', 'lex_liwc_Analytic', 'lex_liwc_Clout',
       'lex_liwc_Authentic', 'lex_liwc_Tone', 'lex_liwc_WPS',
       'lex_liwc_Sixltr', 'lex_liwc_Dic', 'lex_liwc_function',
       ...
       'lex_dal_min_pleasantness', 'lex_dal_min_activation',
       'lex_dal_min_imagery', 'lex_dal_avg_activation', 'lex_dal_avg_imagery',
       'lex_dal_avg_pleasantness', 'social_upvote_ratio',
       'social_num_comments', 'syntax_fk_grade', 'sentiment'],
      dtype='object', length=107)


In [3]:
t = Tokenizer()
t.fit_on_texts(train['processed_text'].values)
vocab_size = len(t.word_index) + 1
print(vocab_size)

8566


In [4]:
encoded_comments_train = t.texts_to_sequences(train['processed_text'])
encoded_comments_test = t.texts_to_sequences(test['processed_text'])

In [5]:
max_length = 512
padded_comments_train = pad_sequences(encoded_comments_train, maxlen=max_length, padding='post')
padded_comments_test = pad_sequences(encoded_comments_test, maxlen=max_length, padding='post')

In [6]:
y_train = train['label'].values
y_test = test['label'].values

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [7]:
w2v_model = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [8]:
# create a weight matrix for words in training docs
embedding_matrix_w2v = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v_model[word]
    except:
        embedding_vector = [0]*300

    if embedding_vector is not None:
        embedding_matrix_w2v[i] = embedding_vector

embedding_matrix_w2v.shape

(8566, 300)

In [9]:
#tensorboard = TensorBoard(log_dir='lstm_model')

In [10]:
reduce_lr = ReduceLROnPlateau(monitor='val_f1_m',
                              mode = 'max',
                              factor=0.5,
                              patience=5,
                              min_lr=0.0001,
                              verbose=10)

checkpoint = ModelCheckpoint("lstm_model.h5",
                               monitor="val_f1_m",
                               mode="max",
                               save_best_only = True,
                               verbose=1)

earlystop = EarlyStopping(monitor = 'val_f1_m',
                            mode="max",
                            min_delta = 0,
                            patience = 5,
                            verbose=1)

In [11]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [12]:
import keras
# class Attention(keras.layers.Layer):
#     def __init__(self, step_dim,
#                  W_regularizer=None, b_regularizer=None,
#                  W_constraint=None, b_constraint=None,
#                  bias=True, **kwargs):
#         self.supports_masking = True
#         self.init = initializers.get('glorot_uniform')
#         self.W_regularizer = regularizers.get(W_regularizer)
#         self.b_regularizer = regularizers.get(b_regularizer)
#         self.W_constraint = constraints.get(W_constraint)
#         self.b_constraint = constraints.get(b_constraint)
#         self.bias = bias
#         self.step_dim = step_dim
#         self.features_dim = 0
#         super(Attention, self).__init__(**kwargs)

#     def build(self, input_shape):
#         assert len(input_shape) == 3
#         self.W = self.add_weight(shape=(input_shape[-1],),
#                                  name='{}_W'.format(self.name),
#                                  initializer=self.init,
#                                  regularizer=self.W_regularizer,
#                                  constraint=self.W_constraint)
#         self.features_dim = input_shape[-1]
#         if self.bias:
#             self.b = self.add_weight((input_shape[1],),
#                                      name='{}_b'.format(self.name),
#                                      initializer='zero',
#                                      regularizer=self.b_regularizer,
#                                      constraint=self.b_constraint)
#         else:
#             self.b = None
#         self.built = True

#     def compute_mask(self, input, input_mask=None):
#         return None

#     def call(self, x, mask=None):
#         features_dim = self.features_dim
#         step_dim = self.step_dim
#         eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
#         if self.bias:
#             eij += self.b
#         eij = K.tanh(eij)
#         a = K.exp(eij)
#         if mask is not None:
#             a *= K.cast(mask, K.floatx())
#         a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
#         a = K.expand_dims(a)
#         weighted_input = x * a
#         return K.sum(weighted_input, axis=1)

#     def compute_output_shape(self, input_shape):
#         return input_shape[0],  self.features_dim
    
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences

        super(attention,self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="normal")

        super(attention,self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:

            return output
        return K.sum(output, axis=1)

In [13]:
model = Sequential()
model.add(Input(shape=(512,), name='main_input'))
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix_w2v], trainable=False))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(100, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(100, return_sequences = True)))
model.add(Dropout(0.3))
model.add(attention(10))
model.add(Flatten())
model.add(Dense(2, activation='softmax', name='fully_connected'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_score])
model.summary()

#keras.utils.vis_utils.pydot = pyd
#plot_model(model, to_file='lstm_model.png')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 300)          2569800   
                                                                 
 dropout (Dropout)           (None, 512, 300)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 512, 200)         241200    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 512, 200)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 512, 200)         181200    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 512, 200)          0

In [14]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=[get_f1])

In [15]:
lstm_model_h1 = model.fit(padded_comments_train, y_train,
               batch_size=64,
               epochs=50,
               verbose=1, callbacks=[checkpoint, earlystop, reduce_lr], #tensorboard
               validation_data=(padded_comments_test, y_test))

Epoch 1/50
45/45 [==============================] - 643s 14s/step - loss: 0.6651 - get_f1: 0.5718 - val_loss: 0.6169 - val_get_f1: 0.6613 - lr: 0.0010
Epoch 2/50
15/45 [=========>....................] - ETA: 6:46 - loss: 0.5583 - get_f1: 0.7156 

KeyboardInterrupt: 

In [ ]:
score_1 = model.evaluate(padded_comments_test, y_test)
print(score_1)

In [ ]:
cnf_mat = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(model.predict(padded_comments_test), axis=1))

print(cnf_mat)
sns.heatmap(cnf_mat, annot=True, fmt='g', linewidths=.5, xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])

    #%%

plt.plot(lstm_model_h1.history['loss'][1:])
plt.plot(lstm_model_h1.history['val_loss'][1:])
plt.title('Model metric')
plt.ylabel('F1 metric')
plt.xlabel('epoch')
plt.legend(['train','Validation'], loc='upper left')
plt.savefig('bilstm_f1_100_100_512.png')
plt.show()

In [ ]:
plt.plot(lstm_model_h1.history['loss'][1:])
plt.plot(lstm_model_h1.history['val_loss'][1:])
plt.title('Model Los')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train','Validation'], loc='upper left')
plt.savefig('Bilstm_Loss_100_100_512.png')
plt.show()

## Model 2: BiLstm + Attention + features

In [ ]:
# def df_to_dataset(dataframe, shuffle=True, batch_size=32):
#     dataframe = dataframe.copy()
#     labels = dataframe.pop('target')
#     ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
#     if shuffle:
#     ds = ds.shuffle(buffer_size=len(dataframe))
#     ds = ds.batch(batch_size)
#     return ds

In [ ]:
# batch_size = 64 #
# features_train = df_to_dataset(features_train, batch_size=batch_size)
# #val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
# features_test = df_to_dataset(features_test, shuffle=False, batch_size=batch_size)

In [ ]:
# Defining the embedding layer
embedding_dim = 300
input1=Input(shape=(512,))

embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix_w2v], trainable=False)(input1)
# Building LSTM for text features

lstm1 = Bidirectional(GRU(100,dropout = 0.3, return_sequences = True))(embedding_layer)
lstm2 = Bidirectional(GRU(100,dropout = 0.3, return_sequences = True))(lstm1)
attention = Attention(10)([lstm1,lstm2])
flatten = Flatten()(attention)

lstm_output =  Model(inputs = input1,outputs = flatten)

#Inputting Number features
input2=Input(shape=(features_len,))

# Merging inputs
merge = concatenate([lstm_output.output,input2])

# Building dense layers for regression with number features
reg_dense1 = Dense(64, activation='relu')(merge)
reg_dense2 = Dense(16, activation='relu')(reg_dense1)
output1 = Dense(2, activation='softmax')(reg_dense2)

# Building dense layers for classification with number features
clf_dense1 = Dense(64, activation='relu')(merge)
clf_dense2 = Dense(16, activation='relu')(clf_dense1)

# 5 Categories in classification
output2 = Dense(2, activation='softmax')(clf_dense2)

model = Model(inputs=[input1,input2], outputs=[output1,output2])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(lr=0.0001), metrics=[get_f1])
model.summary()

In [ ]:
lstm_model_h2 = model.fit([padded_comments_train, features_train.to_numpy()], y_train,
               batch_size=64,
               epochs=50,
               verbose=1, callbacks=[ checkpoint, earlystop, reduce_lr],  #tensorboard
               validation_data=([padded_comments_test, features_test.to_numpy()], y_test))

In [ ]:
score_1 = model.evaluate([padded_comments_test,features_test], y_test)
score_1
print('f1',score_1[3])

In [ ]:
plt.plot(lstm_model_h2.history['loss'][1:])
plt.plot(lstm_model_h2.history['val_loss'][1:])
plt.title('Model Los')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train','Validation'], loc='upper left')
plt.savefig('Bilstm_Loss_100_100_512.png')
plt.show()